**BOUBACAR TOULBA 22019**

In [ ]:
# import des librairies necessaires
import pandas as pd
import numpy as np
# import tensorflow_datasets as tfds
import tensorflow as tf
tf.__version__

'2.15.0'

Import des données

In [ ]:
df = pd.read_csv("/content/fra.txt" , delimiter='\t', header=None)

In [ ]:
# Chargement du document
doc = pd.read_table("fra.txt",header=None)
doc.head()

,0,1
0,Go.,Va !
1,Hi.,Salut !
2,Run!,Cours !
3,Run!,Courez !
4,Wow!,Ça alors !


In [ ]:
#Prenons simplement un sample de 5000 phrases pour eviter des lenteurs
doc = doc.sample(5000)

In [ ]:
#Chargement de langages francais et anglais de spacy
!python -m spacy download fr_core_news_sm
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 39.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 39.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Import de chacun des langages
import fr_core_news_sm
import en_core_web_sm
nlp_fr = fr_core_news_sm.load()
nlp_en = en_core_web_sm.load()

In [ ]:
# Chargement du corpus entier de phrases francaises et anglaises
fr_corpus = " ".join(doc.iloc[:,1].to_list())
en_corpus = " ".join(doc.iloc[:,0].to_list())

In [ ]:
fr_corpus

'Il est déterminé à aller en Angleterre. Je compatis à ta douleur. Elle a les cheveux mouillés. Elle lui demanda de l\'appeler plus tard. Y a-t-il quelqu\'un à qui vous puissiez vous fier ? Je n\'arrive pas à m\'endormir. Vous n\'êtes pas trop en retard. Cette robe a l\'air éblouissante sur toi. J\'étais très riche, jusqu\'à ce que je la rencontre. Génial\u202f! Ils ont fouillé ici et là à la recherche de survivants. Elles se téléphonent presque tous les jours. J\'ai des badges d\'entrée pour les coulisses. Quand pars-tu\u202f? Tom dit qu\'il sait déjà comment faire cela. Cherches-tu quelqu\'un ? Je dois aller au magasin acheter du shampooing et du dentifrice. J\'ai fort bien dormi la nuit passée. Il faisait nuit. Appelle-moi si vous voulez que nous fassions quelque chose ensemble, demain. Vous devez être amoureuses. Personne ne semble avoir prêté attention à ce qu\'il a dit. L\'espagnol est sa langue maternelle. J\'ai besoin de votre réponse avant la fin de la journée. Il est retourné

In [ ]:
# Chargement des deux carpus dans spacy
# %%%time
import time
nlp_fr.max_length = len(fr_corpus)
nlp_en.max_length = len(en_corpus)

fr_doc = nlp_fr(fr_corpus)
en_doc = nlp_en(en_corpus)

In [ ]:
# Tokenisation de chacune des phrases vai spacy
# %%time
doc["fr_tokens"] = doc.iloc[:,1].apply(lambda x: nlp_fr.tokenizer(x))
doc["en_tokens"] = doc.iloc[:,0].apply(lambda x: nlp_en.tokenizer(x))

doc.tail()

,0,1,fr_tokens,en_tokens
85025,Tom looks like an accountant.,Tom ressemble à un comptable.,"(Tom, ressemble, à, un, comptable, .)","(Tom, looks, like, an, accountant, .)"
25026,We swam in the sea.,Nous nageâmes dans la mer.,"(Nous, nageâmes, dans, la, mer, .)","(We, swam, in, the, sea, .)"
39849,Is there someone else?,Y-a-t'il quelqu'un d'autre?,"(Y, -, a, -, t', il, quelqu', un, d', autre, ?)","(Is, there, someone, else, ?)"
118918,No one knew how much Tom loved Mary.,Personne ne savait à quel point Tom aimait Mary.,"(Personne, ne, savait, à, quel, point, Tom, ai...","(No, one, knew, how, much, Tom, loved, Mary, .)"
96759,Tom shouldn't make fun of Mary.,Tom ne devrait pas se moquer de Mary.,"(Tom, ne, devrait, pas, se, moquer, de, Mary, .)","(Tom, should, n't, make, fun, of, Mary, .)"


In [ ]:
# Creation d'un set() qui va prendre tous les tokens de votre corpus
en_tokens = [token.text for token in en_doc]
en_vocabulary_set = set(en_tokens) #retirer tous les doublons
en_vocab_size = len(en_vocabulary_set)
en_vocab_size

3551

In [ ]:
# Meme chose pour le francais
fr_tokens = [token.text for token in fr_doc]
fr_vocabulary_set = set(fr_tokens) # retirer les doublons
fr_vocab_size = len(fr_vocabulary_set)
fr_vocab_size

4864

In [ ]:
# Creation d'un id pour chacun des tokens
all_en_tokens = {}
for i , en_token in enumerate(en_vocabulary_set):
  all_en_tokens[en_token]= i+1 # On prend a i+1 pour laisser la valeur

all_fr_tokens ={}
for i, fr_token in enumerate(fr_vocabulary_set):
  all_fr_tokens[fr_token]=i+1

In [ ]:
# Creation des fonctions qui vont creer un vecteur d'indices pour chacune
def en_tokens_to_index(tokens):
  indices = []
  for token in tokens:
    indices.append(all_en_tokens[token.text])

  return indices

def fr_tokens_to_index(tokens):
  indices=[]
  for token in tokens:
    indices.append(all_fr_tokens[token.text])

  return indices

In [ ]:
# Transformation des tokens en indices
doc["fr_indices"] = doc["fr_tokens"].apply(lambda x: fr_tokens_to_index(x))
doc["en_indices"] = doc["en_tokens"].apply(lambda x: en_tokens_to_index(x))

In [ ]:
doc.tail()

,0,1,fr_tokens,en_tokens,fr_indices,en_indices
85025,Tom looks like an accountant.,Tom ressemble à un comptable.,"(Tom, ressemble, à, un, comptable, .)","(Tom, looks, like, an, accountant, .)","[3572, 2028, 1042, 4653, 126, 1296]","[2564, 698, 1468, 2542, 3231, 939]"
25026,We swam in the sea.,Nous nageâmes dans la mer.,"(Nous, nageâmes, dans, la, mer, .)","(We, swam, in, the, sea, .)","[197, 4073, 4365, 333, 770, 1296]","[1916, 2851, 1936, 886, 2076, 939]"
39849,Is there someone else?,Y-a-t'il quelqu'un d'autre?,"(Y, -, a, -, t', il, quelqu', un, d', autre, ?)","(Is, there, someone, else, ?)","[3664, 711, 4476, 711, 3878, 4813, 2069, 4653,...","[178, 2748, 2615, 1585, 3422]"
118918,No one knew how much Tom loved Mary.,Personne ne savait à quel point Tom aimait Mary.,"(Personne, ne, savait, à, quel, point, Tom, ai...","(No, one, knew, how, much, Tom, loved, Mary, .)","[3073, 1249, 327, 1042, 2319, 4139, 3572, 2864...","[3413, 2619, 2556, 3132, 3098, 2564, 746, 2168..."
96759,Tom shouldn't make fun of Mary.,Tom ne devrait pas se moquer de Mary.,"(Tom, ne, devrait, pas, se, moquer, de, Mary, .)","(Tom, should, n't, make, fun, of, Mary, .)","[3572, 1249, 1385, 3728, 1024, 4667, 1554, 303...","[2564, 689, 3335, 609, 459, 2872, 2168, 939]"


In [ ]:
# Creation d'une fonction qui va compter la longueur maximum d'une phrase
def max_len(lines):
    return max(len(line) for line in lines)

In [ ]:
# Application de la focntion sur les tokens francais et anglais
fr_max_len = max_len(doc['fr_indices'].to_list())
en_max_len = max_len(doc['en_indices'].to_list())

In [ ]:
tf.keras.preprocessing.sequence.pad_sequences?

In [ ]:
#Utilisation de keras pour creer des sequences de tokens de la meme
#%d%time
padded_fr_indices = tf.keras.preprocessing.sequence.pad_sequences(doc["fr_indices"],
                                                                  maxlen=fr_max_len,
                                                                  padding="post")
pad_en_indices = tf.keras.preprocessing.sequence.pad_sequences(doc["en_indices"],
                                                               maxlen=en_max_len,
                                                               padding="post")

In [ ]:
# Visualisation de la shape d'un des tenseurs
padded_fr_indices.shape

(5000, 26)

In [ ]:
# Application de la catégorisation de la variable cible
binarized_en_indices = tf.keras.utils.to_categorical(pad_en_indices, num_classes=en_vocab_size + 1 )

binarized_en_indices.shape

(5000, 24, 3552)

In [ ]:
#Application de la categorisation de la variable cible
en_indices = tf.keras.utils.to_categorical(pad_en_indices,
                                                     num_classes=en_vocab_size+1)
en_indices.shape

(5000, 24, 3552)

In [ ]:
# binarized_en_indices

binarized


In [ ]:
tf.data.Dataset.from_tensor_slices?

In [ ]:
# Creation de tf.data.Dataset pour chacun des tenseurs francais et anglais
fr_ds = tf.data.Dataset.from_tensor_slices(padded_fr_indices)
en_ds = tf.data.Dataset.from_tensor_slices(binarized_en_indices)

In [ ]:
#Create a tensorflow datase complet
tf_ds = tf.data.Dataset.zip(fr_ds,en_ds)

In [ ]:
next(iter(tf_ds))

(<tf.Tensor: shape=(26,), dtype=int32, numpy=
 array([3696, 1552, 1445, 1042, 2813, 4004, 4003, 1296,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0], dtype=int32)>,
 <tf.Tensor: shape=(24, 3552), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)>)

In [ ]:
# Shuffle & Batch
BATCH_SIZE = 32

tf_ds = tf_ds.shuffle(len(doc)).batch(BATCH_SIZE)

In [ ]:
len(doc)/32*0.7

109.375

In [ ]:
# train test split
TAKE_SIZE = int(0.7*len(doc)/BATCH_SIZE)

train_data = tf_ds.take(TAKE_SIZE)
test_data = tf_ds.skip(TAKE_SIZE)

In [ ]:
  tf.keras.layers.Embedding?

In [ ]:
# Creation du modele
model = tf.keras.Sequential([
    # Couche d'Input Word Embedding
    tf.keras.layers.Embedding(fr_vocab_size + 1, 64, mask_zero = True),

    # Couche LSTM Bidirectionnelle
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True)),

    # Nouvelle Couche LSTM Bidirectionnelle
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = False)),

    # Repeat Vector
    tf.keras.layers.RepeatVector(binarized_en_indices.shape[1]),

    # Nouvelle Couche LSTM
    tf.keras.layers.LSTM(32, return_sequences = True),
    #couche de sortie avec le nombre de neurones en sortie egale au nombre de classe avec focntion softmax
     tf.keras.layers.TimeDistributed( tf.keras.layers.Dense(en_vocab_size+1, activation = "softmax"))
    ])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          311360    
                                                                 
 bidirectional (Bidirection  (None, None, 128)         66048     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 repeat_vector (RepeatVecto  (None, 24, 128)           0         
 r)                                                              
                                                                 
 lstm_2 (LSTM)               (None, 24, 32)            20608     
                                                        

In [ ]:
# prediction

# Prediction 'aleatoire' pour tester notre modele
input_text, output_text = next(iter(train_data))
print(input_text.numpy().shape)
print(model.predict(input_text).shape)
print(output_text.numpy().shape)

(32, 26)
1/1 [==============================] - 0s 171ms/step
(32, 24, 3552)
(32, 24, 3552)


In [ ]:
initial_learning_rate = 0.001


In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate ,
    decay_steps=1090 ,
    decay_rate=0.96 ,
    staircase=True )

In [ ]:
# Utilisation d un compileur simple avec un optimiseure Adam pour le calcul

optimizer= tf.keras.optimizers.Adam(
    learning_rate = lr_schedule
)

In [ ]:
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

# Application du modele sur 200 epochs
history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=200)

Epoch 1/200
109/109 [==============================] - 60s 377ms/step - loss: 5.3246 - categorical_accuracy: 0.6772 - val_loss: 2.9717 - val_categorical_accuracy: 0.6837
Epoch 2/200
109/109 [==============================] - 28s 259ms/step - loss: 2.5542 - categorical_accuracy: 0.6837 - val_loss: 2.4858 - val_categorical_accuracy: 0.6766
Epoch 3/200
109/109 [==============================] - 21s 195ms/step - loss: 2.3335 - categorical_accuracy: 0.6843 - val_loss: 2.2308 - val_categorical_accuracy: 0.6824
Epoch 4/200
109/109 [==============================] - 23s 207ms/step - loss: 2.1376 - categorical_accuracy: 0.6849 - val_loss: 2.0759 - val_categorical_accuracy: 0.6841
Epoch 5/200
109/109 [==============================] - 27s 247ms/step - loss: 2.0358 - categorical_accuracy: 0.6836 - val_loss: 2.0048 - val_categorical_accuracy: 0.6829
Epoch 6/200
109/109 [==============================] - 23s 207ms/step - loss: 1.9574 - categorical_accuracy: 0.6853 - val_loss: 1.9329 - val_categoric

In [ ]:









# Test d'une traduction

for input_text , translation in test_data.take(1) :
  fr_text = input_text
  pred = model.predict_classes(inpt_text)
  translation = translation


# phrases en francais




for indice in fr_text[0]:
  for key , value in all_fr_tokens.items() :
    if indice == value :
      print(key)